In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import linear_model

In [2]:
# crime_2011 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2011.csv")
crime_2012 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2012.csv")
crime_2013 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2013.csv")
crime_2014 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2014.csv")
crime_2015 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2015.csv")
crime_2016 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2016.csv")

violence_crime_code = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/Experiment/Violence_crime.csv")
stop_search = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Stop_and_Search__Field_Interviews_.csv")
qol = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/311_Calls__2012-Present_.csv")

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
# crime-related stop & search
stop_search = stop_search[(stop_search.StopDescription == "CRIMINAL VIOLATION") |
                          (stop_search.StopDescription == "PRESENT AT CRIME SCENE") |
                          (stop_search.StopDescription == "SUSPECT PERSON") |
                          (stop_search.StopDescription == "SUSPECT VEHICLE") |
                          (stop_search.StopDescription == "JUVENILE VIOLATION")]

In [17]:
# vilence_crime_2011 = pd.merge(crime_2011, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')
vilence_crime_2012 = pd.merge(crime_2012, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')
vilence_crime_2013 = pd.merge(crime_2013, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')
vilence_crime_2014 = pd.merge(crime_2014, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')
vilence_crime_2015 = pd.merge(crime_2015, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')
vilence_crime_2016 = pd.merge(crime_2016, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')

In [18]:
frames = [vilence_crime_2012, vilence_crime_2013, vilence_crime_2014, vilence_crime_2015, vilence_crime_2016]
crime_whole = pd.concat(frames)

In [10]:
qol = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/311_Calls__2012-Present_.csv")

In [20]:
qol = qol[(qol.issue_type == "Code Enforcement General Request") |
          (qol.issue_type == "Street Light") |
          (qol.issue_type == "Trash/Garbage Pickup") |
          (qol.issue_type == "Residential Recycling Programs") |
          (qol.issue_type == "Abandoned Vehicle Reporting/Removal") |
          (qol.issue_type == "Large Item Trash/Garbage Pickup") |
          (qol.issue_type == "Pothole/Roadway Surface Repair") |
          (qol.issue_type == "General Service Request") |
          (qol.issue_type == "Street Flooding/Drainage") |
          (qol.issue_type == "Illegal Dumping Reporting")]

In [5]:
def getDate(data, dateCol, colUsed):
    newDf = data.copy()
    date = pd.to_datetime(data[dateCol])     
    year = date.dt.year
    month = date.dt.month
    day = date.dt.day
    newDf['Date'] = pd.to_datetime(year*10000 + month*100 + day, format='%Y%m%d')
    totalCols = colUsed + ['Date']
    newDf = newDf[totalCols]
    return newDf

In [6]:
def getYearAndMonth(data):
    newDf = data.copy()
    year = data.Date.dt.year.astype(str)
    month = data.Date.dt.month.astype(str)
    day = np.repeat(1, data.shape[0]).astype(str)
    date = year + '-' + month + '-' + day
    date = pd.to_datetime(date)
    newDf.Date = date
    return newDf

## City-level Analysis

In [22]:
# whole city level analysis
crime_whole_date = getDate(crime_whole, 'TimeCreate', [])
stop_search_date = getDate(stop_search, 'EventDate', [])
qol_date = getDate(qol, 'ticket_created_date_time', ['issue_type'])

In [26]:
stop_search_date = stop_search_date[(stop_search_date['Date'] >= dt.datetime(2012, 1, 1)) & 
                                    (stop_search_date['Date'] <= dt.datetime(2016, 12, 31))]
qol_date = qol_date[(qol_date['Date'] >= dt.datetime(2012, 1, 1)) & 
                                    (qol_date['Date'] <= dt.datetime(2016, 12, 31))]

In [27]:
crime_whole_month = getYearAndMonth(crime_whole_date)
stop_search_month = getYearAndMonth(stop_search_date)
qol_month = getYearAndMonth(qol_date)

In [31]:
crime_agg = pd.DataFrame({"Date": list(crime_whole_month.Date.value_counts().index),
                          "Number of Crimes": crime_whole_month.Date.value_counts().values
                         }, index = None)
crime_agg = crime_agg.sort_values('Date')

stopSearch_agg = pd.DataFrame({"Date": list(stop_search_month.Date.value_counts().index),
                               "Number of SNS": stop_search_month.Date.value_counts().values
                              }, index = None)
stopSearch_agg = stopSearch_agg.sort_values('Date')

In [62]:
def getQolTypeTable(df, issueList, startYr, endYr):
    res = None
    for issue in issueList:
        issueDf = df[df.issue_type == issue]
        issueDf = pd.DataFrame(issueDf['Date'])
        
        date = [str(x) for x in list(issueDf.Date.value_counts().index)]
        val = list(issueDf.Date.value_counts().values)
        mean = int(sum(val) / len(val))
        
        for year in range(startYr, endYr + 1):
            for month in range(1, 13):
                if month < 10: month = "0"+ str(month)
                d = str(year) + "-" + str(month) + '-01 00:00:00'
                if d not in date:
                    date.append(d)
                    val.append(mean)
                        
        date = date = pd.to_datetime(pd.Series(date))  
        issue_agg = pd.DataFrame({"Date": date,
                                  "Number of " + issue: val
                                 }, index = None)
        issue_agg = issue_agg.sort_values('Date')
        
        
        # print(issue_agg)
        
        issue_agg.index = list(range(60))
        if issueList.index(issue) == 0: res = issue_agg
        else: 
            res = pd.concat([res, issue_agg.iloc[:, 1:]], axis=1)
    return res

In [63]:
issueList = list(qol.issue_type.value_counts().index)
qolTable = getQolTypeTable(qol_month, issueList, 2012, 2016)

In [64]:
qolTable

,Date,Number of Code Enforcement General Request,Number of Street Light,Number of Trash/Garbage Pickup,Number of Residential Recycling Programs,Number of Abandoned Vehicle Reporting/Removal,Number of Large Item Trash/Garbage Pickup,Number of Pothole/Roadway Surface Repair,Number of General Service Request,Number of Street Flooding/Drainage,Number of Illegal Dumping Reporting
0,2012-01-01,532,508,380,318,296,224,223,156,150,132
1,2012-02-01,532,508,380,318,296,224,223,156,150,132
2,2012-03-01,309,216,13,10,15,4,79,156,131,6
3,2012-04-01,672,512,40,33,132,17,1263,153,1122,17
4,2012-05-01,871,374,60,52,189,29,983,75,391,33
5,2012-06-01,707,340,37,17,165,19,158,51,138,27
6,2012-07-01,779,383,130,101,196,38,197,44,279,29
7,2012-08-01,711,463,210,163,165,97,220,54,386,27
8,2012-09-01,485,1201,578,178,209,309,221,64,251,46
9,2012-10-01,580,1090,308,223,212,182,220,75,190,45


In [68]:
crime_agg.index = list(range(60))
stopSearch_agg.index = list(range(60))
crime_sns_qol_11to16 = pd.concat([crime_agg, stopSearch_agg["Number of SNS"], qolTable.iloc[:, 1:]], axis=1)
crime_sns_qol_11to16.to_csv("crime_sns_qol_11to16.csv")

## Using 2017 crimes as the holding out data

In [38]:
crime_2017 = pd.read_csv("D:/Heinz College/2017 Spring/Capstone Project/time series/Calls_for_Service_2017.csv")
vilence_crime_2017 = pd.merge(crime_2017, violence_crime_code, how = 'inner', left_on = 'Type_', right_on = 'Code')
crime_70119_2017 = vilence_crime_2017[vilence_crime_2017.Zip == 70119]

In [39]:
crime_2017_date = getDate(vilence_crime_2017, 'TimeCreate', [])
crime_70119_2017_date = getDate(crime_70119_2017, 'TimeCreate', [])

In [40]:
crime_2017_month = getYearAndMonth(crime_2017_date)
crime_70119_2017_month = getYearAndMonth(crime_70119_2017_date)

crime_2017_agg = pd.DataFrame({"Date": list(crime_2017_month.Date.value_counts().index),
                               "Number of Crimes": crime_2017_month.Date.value_counts().values
                              }, index = None)
crime_2017_agg = crime_2017_agg.sort('Date')

crime_70119_2017_agg = pd.DataFrame({"Date": list(crime_70119_2017_month.Date.value_counts().index),
                               "Number of Crimes": crime_70119_2017_month.Date.value_counts().values
                              }, index = None)
crime_70119_2017_agg = crime_70119_2017_agg.sort('Date')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [87]:
crime_2017_agg

,Date,Number of Crimes
0,2017-01-01,438
1,2017-02-01,390
2,2017-03-01,303
3,2017-04-01,148


In [41]:
crime_70119_2017_agg

,Date,Number of Crimes
0,2017-01-01,68
2,2017-02-01,43
1,2017-03-01,43
3,2017-04-01,24


## Zone-level Analysis (Zipcode = 70119)

In [76]:
# zipcode-level analysis
crime_70119 = crime_whole[crime_whole.Zip == 70119]
sns_70119 = stop_search[stop_search.Zip == 70119]
qol_70119 = qol[qol.zip_code == 70119]


crime_70119_date = getDate(crime_70119, 'TimeCreate', [])
sns_70119_date = getDate(sns_70119, 'EventDate', [])
qol_70119_date = getDate(qol_70119, 'ticket_created_date_time', ['issue_type'])

In [77]:
sns_70119_date = sns_70119_date[(sns_70119_date['Date'] >= dt.datetime(2012, 1, 1)) & 
                                (sns_70119_date['Date'] <= dt.datetime(2016, 12, 31))]
qol_70119_date = qol_70119_date[(qol_70119_date['Date'] >= dt.datetime(2012, 1, 1)) & 
                                    (qol_70119_date['Date'] <= dt.datetime(2016, 12, 31))]

In [78]:
crime_70119_month = getYearAndMonth(crime_70119_date)
sns_70119_month = getYearAndMonth(sns_70119_date)
qol_70119_month = getYearAndMonth(qol_70119_date)

In [79]:
crime_70119_agg = pd.DataFrame({"Date": list(crime_70119_month.Date.value_counts().index),
                                "Number of Crimes": crime_70119_month.Date.value_counts().values
                               }, index = None)
crime_70119_agg = crime_70119_agg.sort_values('Date')

sns_70119_agg = pd.DataFrame({"Date": list(sns_70119_month.Date.value_counts().index),
                              "Number of SNS": sns_70119_month.Date.value_counts().values
                             }, index = None)
sns_70119_agg = sns_70119_agg.sort_values('Date')

In [80]:
issueList_70119 = list(qol_70119.issue_type.value_counts().index)
qolTable_70119 = getQolTypeTable(qol_70119_month, issueList_70119, 2012, 2016)

In [84]:
crime_70119_agg.index = list(range(60))
sns_70119_agg.index = list(range(sns_70119_agg.shape[0]))

crime_sns_70119_11to16 = pd.merge(crime_70119_agg, sns_70119_agg, how = 'left', on = 'Date')
crime_sns_70119_11to16 = crime_sns_70119_11to16.fillna(crime_sns_70119_11to16["Number of SNS"].median())
crime_sns_qol_70119_11to16 = pd.concat([crime_sns_70119_11to16, qolTable_70119.iloc[:, 1:]], axis=1)
# crime_sns_70119_11to16 = pd.concat([crime_70119_agg, sns_70119_agg["Number of SNS"]], axis=1)
crime_sns_qol_70119_11to16.to_csv("crime_sns_qol_70119_11to16.csv")

## Data Prep for  Regression

### whole data

In [69]:
# create lag variables
def getLagTable(data, number):
    newData = data.copy()
    
    for num in range(1, number + 1):
        lagVar = data.iloc[:data.shape[0] - num, 1:]
        lagVar = pd.Series([-1] * num).append(lagVar)
        lagVar.index = data.index
        # print(lagVar)
        name = "lag" + str(num)
        newData[name] = lagVar.iloc[:, 1:]
    
    return newData        

In [72]:
sns_lag_whole = getLagTable(stopSearch_agg, 12)

crime_sns_lag = pd.merge(crime_agg, sns_lag_whole, how = 'left', on = 'Date')
# crime_sns_70119_11to16 = pd.concat([crime_70119_agg, sns_70119_agg["Number of SNS"]], axis=1)
# crime_sns_70119_11to16.to_csv("crime_sns_70119_11to16.csv")
# sns_lag_whole
crime_sns_lag['Month'] = crime_sns_lag.Date.dt.month
crime_sns_lag['Year'] = crime_sns_lag.Date.dt.year
final_whole_reg = crime_sns_lag.iloc[12:, 1:]
final_whole_reg.to_csv("crime_effsns_lag.csv")

In [71]:
clf = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
X = final_whole_reg.drop(["Number of Crimes"], axis = 1)
y = final_whole_reg["Number of Crimes"]
clf.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Zipcode 70119

In [89]:
sns_70119_agg = crime_sns_70119_11to16[["Date", "Number of SNS"]]

sns_lag_70119 = getLagTable(sns_70119_agg, 12)

crime_sns_lag_70119 = pd.merge(crime_70119_agg, sns_lag_70119, how = 'left', on = 'Date')
crime_sns_lag_70119['Month'] = crime_sns_lag_70119.Date.dt.month
crime_sns_lag_70119['Year'] = crime_sns_lag_70119.Date.dt.year
# crime_sns_70119_11to16 = pd.concat([crime_70119_agg, sns_70119_agg["Number of SNS"]], axis=1)
# crime_sns_70119_11to16.to_csv("crime_sns_70119_11to16.csv")
# sns_lag_whole
final_70119_reg = crime_sns_lag_70119.iloc[12:, 1:]
final_70119_reg.to_csv("crime_effsns_lag_70119.csv")

In [86]:
final_70119_reg

,Number of Crimes,Number of SNS,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,lag10,lag11,lag12,Month,Year
12,36,48,49.0,65.0,50.0,45.0,83.0,78.0,99.0,99.0,81.0,125.0,83.0,83.0,1,2013
13,41,29,48.0,49.0,65.0,50.0,45.0,83.0,78.0,99.0,99.0,81.0,125.0,83.0,2,2013
14,41,46,29.0,48.0,49.0,65.0,50.0,45.0,83.0,78.0,99.0,99.0,81.0,125.0,3,2013
15,47,34,46.0,29.0,48.0,49.0,65.0,50.0,45.0,83.0,78.0,99.0,99.0,81.0,4,2013
16,38,42,34.0,46.0,29.0,48.0,49.0,65.0,50.0,45.0,83.0,78.0,99.0,99.0,5,2013
17,29,42,42.0,34.0,46.0,29.0,48.0,49.0,65.0,50.0,45.0,83.0,78.0,99.0,6,2013
18,34,75,42.0,42.0,34.0,46.0,29.0,48.0,49.0,65.0,50.0,45.0,83.0,78.0,7,2013
19,35,63,75.0,42.0,42.0,34.0,46.0,29.0,48.0,49.0,65.0,50.0,45.0,83.0,8,2013
20,39,65,63.0,75.0,42.0,42.0,34.0,46.0,29.0,48.0,49.0,65.0,50.0,45.0,9,2013
21,30,61,65.0,63.0,75.0,42.0,42.0,34.0,46.0,29.0,48.0,49.0,65.0,50.0,10,2013
